In [41]:
#!pip install qpsolvers
#!pip install cvxopt

In [42]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg
from tqdm import tqdm
import numba 
from numba import njit, vectorize, jit
import time 
import scipy

import qpsolvers
from qpsolvers import solve_qp
from qpsolvers import dense_solvers, sparse_solvers

from scipy import optimize

import cvxopt
import cvxopt.solvers

In [2]:
X_train = pd.read_csv('data/Xtr0_mat100.csv',delimiter= ' ', header= None)
Y_train = pd.read_csv('data/Ytr0.csv',delimiter= ',')
X_test = pd.read_csv('data/Xte0_mat100.csv',delimiter= ' ', header= None)

In [3]:
X = X_train.values
Y = Y_train['Bound'].to_numpy()

In [4]:
print(Y.shape)

(2000,)


In [5]:
@njit
def GaussianKernel(x, y, sig2 = 1): 
    return np.exp(-numpy.linalg.norm(x-y)**2/(2*sig2))

In [6]:
time1 = time.time()
Kernel = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : GaussianKernel(x2, x1), 1, X), 1, X)
time2 = time.time()
print(time2 - time1, 's')

20.47282576560974 s


In [7]:
print(Kernel)

[[1.         0.98475818 0.98371162 ... 0.98848832 0.98942306 0.98895558]
 [0.98475818 1.         0.9799995  ... 0.98545651 0.98173781 0.97988373]
 [0.98371162 0.9799995  1.         ... 0.97826428 0.98196982 0.97699373]
 ...
 [0.98848832 0.98545651 0.97826428 ... 1.         0.98708787 0.98545651]
 [0.98942306 0.98173781 0.98196982 ... 0.98708787 1.         0.98650493]
 [0.98895558 0.97988373 0.97699373 ... 0.98545651 0.98650493 1.        ]]


Mon SVM en utilisant cvxopt :

In [161]:
#avec cvxopt
class SVM:
    def __init__(self, kernel = GaussianKernel, C = 1):
        self.kernel = kernel
        self.C = C
        self.alpha = None
        self.support_vectors = None
        self.support_Y = None
        
        
    def fit(self, X, Y):
        
        K = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : self.kernel(x2, x1), 1, X), 1, X)
        
        n = len(Y)
        lbd = 1
        #C = 1 / (2 * n * lbd)  #ça dépend si on veut gérer C ou lambda

        label = 2 * Y - 1
        
        P = cvxopt.matrix(np.outer(label, label) * K)
        q = cvxopt.matrix(-np.ones(n))
        A = cvxopt.matrix(label, (1,n), 'd')
        b = cvxopt.matrix(0.0)
                
        '''Je réécris l'inégalité : 0<=y_i*alpha_i<=C avec C = 1/(2*lambda*n)
        comme: G*alpha<=h avec G=stack(diag(Y),-diag(Y)) et h= [C, ..., C, 0, ..., 0] (n fois C et n fois 0)
        ça revient au même et je crois que le solver devrait fonctionner avec ça, mais j'y arrive pas encore
        '''
        
        G1 = np.eye(n)
        G2 = -np.eye(n)
        G = cvxopt.matrix(np.vstack((G1, G2)))
        
        h = np.ones(2*n)
        h[:n] = h[:n] * self.C
        h[n:] = h[:n] * 0
        h = cvxopt.matrix(h)

        solver = cvxopt.solvers.qp(P, q, G, h, A, b)
        
        self.alpha = np.ravel(solver['x'])
        
        #Je retire les vecteurs avec un alpha trop petit
        eps = 1e-5
        supportIndices = self.alpha > eps
        ind = np.arange(n)[supportIndices]
        
        self.support_vectors = X[supportIndices]
        self.support_Y = label[supportIndices]
        self.alpha_ = self.alpha[supportIndices]  #alpha_ : alpha sans les alpha < eps
        
        #Bias
        self.b = 0
        for i in range(len(self.alpha_)):
            self.b = self.support_Y[i]
            self.b -= np.sum( self.alpha_ * self.support_Y * K[ind[i], supportIndices])
        self.b /= len(self.alpha_)
    
    def predict(self, X):
        
        y_predict = np.zeros(len(X))
        
        for i in range(len(X)):
            for alpha, sv, label in zip(self.alpha_, self.support_vectors, self.support_Y):
                y_predict[i] += alpha * label * self.kernel(sv, X[i]) 
        
        return ((y_predict + self.b) > 0)*1
        #return y_predict + self.b

In [159]:
model = SVM(kernel = GaussianKernel, C=0.1)

model.fit(x_train, y_train)

results = model.predict(x_val)

     pcost       dcost       gap    pres   dres
 0: -1.0137e+03 -4.0756e+02  9e+03  2e+01  1e-14
 1: -1.5107e+02 -3.8587e+02  2e+02  3e-02  1e-14
 2: -1.7298e+02 -1.9832e+02  3e+01  3e-03  3e-15
 3: -1.8166e+02 -1.8493e+02  3e+00  4e-04  2e-15
 4: -1.8174e+02 -1.8236e+02  6e-01  7e-05  2e-15
 5: -1.8175e+02 -1.8235e+02  6e-01  7e-05  2e-15
 6: -1.8177e+02 -1.8230e+02  5e-01  5e-05  2e-15
 7: -1.8180e+02 -1.8220e+02  4e-01  3e-05  2e-15
 8: -1.8185e+02 -1.8211e+02  3e-01  2e-05  2e-15
 9: -1.8187e+02 -1.8206e+02  2e-01  9e-06  2e-15
10: -1.8190e+02 -1.8201e+02  1e-01  5e-06  2e-15
11: -1.8192e+02 -1.8198e+02  6e-02  2e-06  2e-15
12: -1.8193e+02 -1.8196e+02  3e-02  8e-07  2e-15
13: -1.8194e+02 -1.8195e+02  1e-02  3e-07  2e-15
14: -1.8194e+02 -1.8194e+02  4e-03  5e-08  2e-15
15: -1.8194e+02 -1.8194e+02  3e-04  2e-09  3e-15
16: -1.8194e+02 -1.8194e+02  6e-06  4e-11  3e-15
Optimal solution found.


In [160]:
results = model.predict(x_val)
print( results, y_val)
n = results == y_val
print(sum(n))
print(model.alpha, len(model.alpha), model.b)

[ 2.47871474e-02  5.62695674e-03  1.59834910e-02  3.39109974e-02
  2.56334315e-02  1.84628201e-02  7.05452996e-03  1.80195657e-02
  6.35307748e-03  3.84969199e-02  3.00141034e-02 -1.43363051e-03
  1.86534584e-02  1.26620410e-02  2.68064349e-02  2.55766623e-02
  2.13360297e-02  6.28488390e-03 -2.64287183e-05  2.28913310e-02
  3.01876749e-03  3.60689510e-02  8.51720492e-03  2.31135754e-02
  2.81516755e-02  6.97781367e-03  1.82406364e-02  1.71836251e-02
  2.73907648e-02  3.27070155e-02  1.50254158e-02  3.30687548e-02
  1.33423447e-02  1.51911426e-02  5.15250809e-03  1.24771833e-02
  1.41825238e-02  3.90248200e-03  4.46219691e-02  2.85429507e-02
 -3.81395657e-03  4.77198548e-02  8.66265445e-03  2.10738007e-02
  9.42223256e-03  1.62191625e-02  3.15892377e-02  3.47096400e-02
  2.18971265e-02  2.94239543e-02  3.88295504e-03  1.88982425e-02
  2.45447991e-02  3.33438323e-03  2.66616455e-02  1.29805436e-02
  1.50068705e-03  1.65158424e-02  2.55359616e-02  2.09004483e-02
  9.67594457e-03  1.58178

En dessous y a mon ancien svm avec qp_solver et un svm sur lequel je me suis aidé avec cvxopt

In [162]:
#Avec qp_solver

class SVM:
    def __init__(self, kernel): #, C):
        self.kernel = kernel
        #self.C = C
        self.alpha = None
        self.support_vectors = None
        self.support_Y = None
        
        
    def fit(self, X, Y):
        
        K = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : self.kernel(x2, x1), 1, X), 1, X)
        
        n = len(Y)
        lbd = 1
        C = 1 #/ (2 * n * lbd)  #ça dépend si on veut gérer C ou lambda
        #C = self.C
        label = 2 * Y - 1
        
        q = - label
        
        '''Je réécris l'inégalité : 0<=y_i*alpha_i<=C avec C = 1/(2*lambda*n)
        comme: G*alpha<=h avec G=stack(diag(Y),-diag(Y)) et h= [C, ..., C, 0, ..., 0] (n fois C et n fois 0)
        ça revient au même et je crois que le solver devrait fonctionner avec ça, mais j'y arrive pas encore
        '''
        G = np.vstack((np.diag(label), -np.diag(label)))

        h = np.ones(2*n)
        h[:n] = h[:n] * C
        h[n:] = h[:n] * 0

        #Si j'ai bien compris mettre un vecteur de 1 pour A et 1 pour b permet de pas les prendre en compte 
        #(ici on a pas d'égalité)
        #A = np.ones(n)
        #b = np.array([1.])

        self.alpha = solve_qp(0.5*K, q.astype('double'), G.astype('double'), h, solver = 'quadprog')
        
        #Pour le moment je garde tout X mais faudrait retirer les X dont le alpha est trop bas
        eps = 1e-12
        supportIndices = self.alpha > eps
        
        self.support_vectors = X[supportIndices]
        self.support_Y = label[supportIndices]
        self.alpha = self.alpha[supportIndices]
    
    def predict(self, X):
        
        def predict_one(x):
            pred = np.apply_along_axis(lambda s: self.kernel(s, x), 1, self.support_vectors)
            pred = pred * self.support_Y * self.alpha
            return np.sum(pred)
        
        preds = np.apply_along_axis(predict_one, 1, X)
        return 1 * (preds > 0)
        
        

In [163]:
x_train = X[:1900]
y_train = Y[:1900]
x_val = X[1900:]
y_val = Y[1900:]

svm = SVM(kernel = GaussianKernel)
svm.fit(x_train, y_train)

results = svm.predict(x_val)

In [91]:
print(results, y_val)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [1 0 0 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0
 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 1
 1 0 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]


En dessous je teste mon svm avant de modifier la classe (pour pas relancer le kernel à chaque fois)

In [70]:
print(qpsolvers.available_solvers)
print(2*(Y_bis) - 1)

['quadprog']
[-1  1  1 ... -1 -1 -1]


In [32]:
Y_bis = 2*y_train - 1 #diminue la dimension de (n,1) à (n,)
X_bis = x_train

#K_bis = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : GaussianKernel(x2, x1), 1, X_bis), 1, X_bis)

n = len(Y_bis)
lbd = 1
C = 1 #/ (2 * n * lbd)

'''Je réécris l'inégalité : 0<=y_i*alpha_i<=C avec C = 1/(2*lambda*n)
comme: G*alpha<=h avec G=stack(diag(Y),-diag(Y)) et h= [C, ..., C, 0, ..., 0] (n fois C et n fois 0)
ça revient au même et je crois que le solver devrait fonctionner avec ça, mais j'y arrive pas encore
'''
G = np.vstack((np.diag(Y_bis), -np.diag(Y_bis)))

h = np.ones(2*n)
h[:n] = h[:n] * C
h[n:] = h[:n] * 0

#Si j'ai bien compris mettre un vecteur de 1 pour A et 1 pour b permet de pas les prendre en compte 
#(ici on a pas d'égalité)
A = np.ones(n)
b = np.array([1.])


time1 = time.time()
tentative = solve_qp(K_bis.astype('double'), Y_bis.astype('double'), G.astype('double'), h, solver = 'quadprog')

time2 = time.time()
print(time2 - time1, 's')


for i in range(len(tentative)):
    print(tentative[i])

76.22652673721313 s
-9.115683808479806e-14
1.5665938513567446e-11
-1.8063990676886445e-10
6.718736752817186e-13
2.63334297714487e-13
5.155429263466893e-15
-1.345559596201473e-10
5.161792524875126e-11
-5.2116148224874e-15
-2.4696695472280675e-13
-3.7002238630356505e-15
4.717799099334883e-11
-4.5327890876304184e-11
-9.38510062977516e-13
1.0153409971868267e-10
-1.2179357314045321e-15
-4.217632613656017e-12
3.87702586802821e-14
1.657759831192873e-11
-1.2078881527567086e-10
8.360427584406006e-15
-5.737856759020523e-11
-1.826663907569626e-11
-1.777476946456024e-13
-6.744107274309603e-15
-4.99576660297522e-10
2.7075988867288465e-15
-3.532359950648739e-11
8.003035892546244e-14
1.3169464513246814e-11
-3.649943234455835e-15
1.2681303187381914e-12
1.4576317189950378e-13
6.296929917970776e-12
1.4167881469028046e-12
-7.624652509617112e-15
-2.0500455044257336e-10
5.4834322181836794e-12
3.174252819362582e-15
-4.544903919335947e-11
-2.1781026889556815e-10
-2.963763910747677e-11
1.871897617210832e-11
8

1.5545596438602205e-13
-6.080061113382203e-11
-1.7388690720026334e-12
-4.487809322926892e-12
2.7582531930443993e-11
1.5803560950280295e-13
1.6623070313884622e-11
-1.1195468509243915e-11
-9.008518868592296e-12
-2.146170371881165e-13
-8.788489597161793e-13
-8.120052774941245e-11
-4.819148923794863e-15
-8.720215349423703e-13
-3.0352256014822714e-12
2.922659226270843e-10
1.1533325726404479e-12
-9.91655784247597e-13
-1.785299012306448e-12
-1.3688931982088016e-12
3.258655154165535e-14
-1.0134082957094456e-13
1.037392071865242e-10
6.203115935092648e-11
-2.468077150415309e-11
4.6296217811425087e-14
-1.523031778866892e-12
1.9759818485144593e-13
1.4499225615930872e-10
3.508540107292358e-11
-1.5524167365294089e-10
1.685547671769403e-11
6.196703832392954e-14
-2.737447612996797e-11
5.1905188327933525e-15
-1.0728128547807672e-14
-4.4767161004063333e-13
-2.9522697068567777e-12
-9.901678745427052e-13
2.0844773019574846e-12
1.026029310087542e-12
-2.194375259980039e-12
-3.517357143252578e-11
-2.80721559

In [47]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

In [141]:
class SVM_git(object):

    def __init__(self, kernel=GaussianKernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples),'d')
        b = cvxopt.matrix(0.0)

        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))

        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        a = np.ravel(solution['x'])

        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == linear_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else:
            self.w = None

    def project(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [142]:
model2 = SVM_git(kernel = GaussianKernel, C=0.1)

model2.fit(x_train, (2*y_train)-1 )

results = model2.predict(x_val)

     pcost       dcost       gap    pres   dres
 0: -1.0137e+03 -4.0756e+02  9e+03  2e+01  1e-14
 1: -1.5107e+02 -3.8587e+02  2e+02  3e-02  1e-14
 2: -1.7298e+02 -1.9832e+02  3e+01  3e-03  2e-15
 3: -1.8166e+02 -1.8493e+02  3e+00  4e-04  2e-15
 4: -1.8174e+02 -1.8236e+02  6e-01  7e-05  2e-15
 5: -1.8175e+02 -1.8235e+02  6e-01  7e-05  2e-15
 6: -1.8177e+02 -1.8230e+02  5e-01  5e-05  2e-15
 7: -1.8180e+02 -1.8220e+02  4e-01  3e-05  2e-15
 8: -1.8185e+02 -1.8211e+02  3e-01  2e-05  2e-15
 9: -1.8187e+02 -1.8206e+02  2e-01  9e-06  2e-15
10: -1.8190e+02 -1.8201e+02  1e-01  5e-06  2e-15
11: -1.8192e+02 -1.8198e+02  6e-02  2e-06  2e-15
12: -1.8193e+02 -1.8196e+02  3e-02  8e-07  2e-15
13: -1.8194e+02 -1.8195e+02  1e-02  3e-07  2e-15
14: -1.8194e+02 -1.8194e+02  4e-03  5e-08  2e-15
15: -1.8194e+02 -1.8194e+02  3e-04  2e-09  3e-15
16: -1.8194e+02 -1.8194e+02  6e-06  4e-11  3e-15
Optimal solution found.
1827 support vectors out of 1900 points
-0.020502816483377862


In [122]:
print( (results+1)/2, y_val)
n = (results+1)/2 == y_val
print(sum(n))
print(model2.a, len(model2.a))

[1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1.
 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 1. 0. 0.] [1 0 0 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0
 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 1
 1 0 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
56
[0.09999995 0.1        0.1        ... 0.0999999  0.09999993 0.1       ] 1827


In [132]:
#Tests sur les fct de prédiction

def predict_one(x):
    pred = np.apply_along_axis(lambda s: GaussianKernel(s, x), 1, X_bis)
    pred = pred * Y_bis * model.alpha
    return np.sum(pred)

def f_from_alpha(alpha, Kernel, X):
    return  lambda x : np.sum([alpha[i]*Kernel(X[i,:],x) for i in range(X.shape[0])])
f_alpha = f_from_alpha(model.alpha, GaussianKernel, x_train)

res = [f_alpha(x_val[i]) for i in range(len(y_val))]

preds = np.apply_along_axis(predict_one, 1, x_val)
#preds = 1 * (preds > 0)
print(preds, y_val, res)

[ 0.02414692  0.00498681  0.01534296  0.03327065  0.02499289  0.01782278
  0.00641423  0.01737927  0.00571243  0.03785679  0.02937389 -0.0020743
  0.01801311  0.01202195  0.02616614  0.02493651  0.02069579  0.00564455
 -0.00066408  0.02225113  0.00237909  0.03542867  0.00787691  0.02247312
  0.02751154  0.00633758  0.01760011  0.0165434   0.0267509   0.03206684
  0.01438477  0.03242849  0.01270201  0.01455076  0.00451208  0.01183673
  0.01354231  0.00326173  0.04398204  0.02790272 -0.00445432  0.04707981
  0.00802263  0.02043321  0.00878177  0.01557869  0.03094886  0.03406918
  0.02125684  0.02878366  0.00324311  0.01825782  0.02390436  0.00269406
  0.02602237  0.01234005  0.00085998  0.0158754   0.02489576  0.02026007
  0.00903557  0.0151773   0.03625054  0.01268538  0.00022868  0.00617082
  0.0134089   0.01244844 -0.00563212  0.00012831  0.02412231  0.02177587
  0.01969552  0.03197964  0.01770224  0.01614737  0.01016502  0.03356141
  0.00210028  0.02871426  0.00812573 -0.00372503  0.

### Ancien pour Ridge et test:

In [45]:
lbd = 3

#w_pas_benef = X.T.dot(np.linalg.inv(X.dot(X.T) + lbd * np.eyes(2000))).dot(Y)
def KRR(X,Y):
    n, d = x.shape
    w = np.linalg.inv(X.T.dot(X) + lbd * n * np.eye(d)).dot(X.T.dot(Y))
    return w


result = X@w_benef